In [1]:
import gdown
import zipfile

url = 'https://drive.google.com/uc?id=1h3WmTxwDAKy2eRw0SCXcNVFdreuquRQN'
output = 'new_data.zip'
gdown.download(url, output, quiet=False)

# Unzip the new_data.zip file
with zipfile.ZipFile("new_data.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")

Downloading...
From (original): https://drive.google.com/uc?id=1h3WmTxwDAKy2eRw0SCXcNVFdreuquRQN
From (redirected): https://drive.google.com/uc?id=1h3WmTxwDAKy2eRw0SCXcNVFdreuquRQN&confirm=t&uuid=7da336f7-ef0d-407b-8055-c38777b1dfbb
To: /kaggle/working/new_data.zip
100%|██████████| 1.23G/1.23G [00:09<00:00, 127MB/s] 


In [2]:
## Import libraries
import os

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from PIL import Image
from skimage.measure import label, regionprops
from sklearn.metrics import (
    accuracy_score,
)
from torch import nn, optim
from imblearn.over_sampling import RandomOverSampler
from torch.utils.data import DataLoader, ConcatDataset, Subset
from torchvision import datasets, transforms, models
from tqdm import tqdm
from collections import Counter


def resize_image(image, target_size=(224, 224)):
    """Resizes an input image to the specified target size.

    Args:
        image (PIL Image or Torch Tensor): Input image to be resized.
        target_size (tuple): Desired output size (height, width).

    Returns:
        PIL Image: Resized image.

    """
    transform = transforms.Resize(target_size)
    if isinstance(image, torch.Tensor):
        image = transforms.ToPILImage()(image)
    return transform(image)


def histogram_equalization(image):
    """Performs histogram equalization on a grayscale image and returns a tensor.

    Args:
        image (PIL Image or Torch Tensor): Input grayscale image.

    Returns:
        torch.Tensor: Equalized image as a tensor.

    """
    # Convert the image to a NumPy array if it's a PIL Image
    if isinstance(image, Image.Image):
        image = np.array(image.convert("L"))  # Convert to grayscale
    elif isinstance(image, torch.Tensor):
        image = image.numpy()

    # Ensure the image is single-channel and np.uint8
    if image.ndim == 3 and image.shape[0] == 1:  # If shape is (1, H, W)
        image = image.squeeze(0)  # Remove the extra channel dimension
    elif image.ndim == 3 and image.shape[-1] == 1:  # If shape is (H, W, 1)
        image = image.squeeze(-1)

    if image.dtype != np.uint8:
        image = image.astype(np.uint8)

    equalized_image = cv2.equalizeHist(image)
    equalized_pil_image = Image.fromarray(equalized_image)
    # equalized_tensor = torch.from_numpy(equalized_image).float().unsqueeze(0)  # Add channel dimension for grayscale
    return equalized_pil_image


def gaussian_blur(image, kernel_size=(5, 5), sigma=0):
    """Applies Gaussian blur to a grayscale image.

    Args:
        image (PIL Image or Torch Tensor): Input grayscale image.
        kernel_size (tuple): Size of the Gaussian kernel.
        sigma (float): Standard deviation for Gaussian kernel.
                       If 0, it will be calculated based on the kernel size.

    Returns:
        torch.Tensor: Blurred image as a tensor.

    """  # noqa: D401
    # Convert to NumPy array if the image is a PIL Image
    if isinstance(image, Image.Image):
        image = np.array(image)
    elif isinstance(image, torch.Tensor):
        # Convert to NumPy if the input is a tensor
        image = image.numpy()

    # If the image has a channel dimension (1, H, W), squeeze it to (H, W)
    if image.ndim == 3 and image.shape[0] == 1:
        image = np.squeeze(image, axis=0)

    blurred_image = cv2.GaussianBlur(image, kernel_size, sigma)
    blurred_tensor = (
        torch.from_numpy(blurred_image).float().unsqueeze(0)
    )  # Add channel dimension for grayscale

    return blurred_tensor


def bilateral_filter(image, diameter=5, sigma_color=75, sigma_space=75):
    """Applies a bilateral filter to a grayscale image.

    Args:
        image (PIL Image, NumPy array, or Torch Tensor): Input grayscale image.
        diameter (int): Diameter of each pixel neighborhood used in the filter.
        sigma_color (float): Filter sigma in the color space.
        sigma_space (float): Filter sigma in the coordinate space.

    Returns:
        torch.Tensor: Filtered image as a tensor.

    """  # noqa: D401
    # Convert to NumPy array if the image is a PIL Image
    if isinstance(image, Image.Image):
        image = np.array(image)
    elif isinstance(image, torch.Tensor):
        image = image.numpy()

    # If the image has a channel dimension (1, H, W), squeeze it to (H, W)
    if image.ndim == 3 and image.shape[0] == 1:
        image = np.squeeze(image, axis=0)

    if image.dtype != np.uint8:
        image = (255 * (image - image.min()) / (image.max() - image.min())).astype(
            np.uint8
        )

    # Apply bilateral filter using OpenCV
    filtered_image = cv2.bilateralFilter(image, diameter, sigma_color, sigma_space)

    # Convert back to a PyTorch tensor
    filtered_tensor = (
        torch.from_numpy(filtered_image).float().unsqueeze(0)
    )  # Add back channel dimension for grayscale

    return filtered_tensor


def adaptive_masking(image, closing_kernel_size=(5, 5)):
    """Applies adaptive masking by removing the diaphragm from a grayscale image.

    Args:
        image (PIL Image, NumPy array, or Torch Tensor): Input grayscale image.
        closing_kernel_size (tuple): Size of the structuring element for morphological closing.

    Returns:
        torch.Tensor: Image with diaphragm removed as a tensor.

    """
    # Convert to NumPy array if the image is a PIL Image
    if isinstance(image, Image.Image):
        image = np.array(image.convert("L"))  # Ensure grayscale
    elif isinstance(image, torch.Tensor):
        # Convert to NumPy if the input is a tensor
        image = image.numpy()

    # If the image has a channel dimension (1, H, W), squeeze it to (H, W)
    if image.ndim == 3 and image.shape[0] == 1:
        image = np.squeeze(image, axis=0)

    # Step 1: Find max and min intensity values
    min_intensity = np.min(image)
    max_intensity = np.max(image)

    # Step 2: Calculate threshold using the formula: threshold = min + 0.9 * (max - min)
    threshold_value = min_intensity + 0.9 * (max_intensity - min_intensity)

    # Step 3: Apply binary thresholding
    _, binary_mask = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)

    # Step 4: Label connected regions and keep only the largest region
    labeled_mask = label(binary_mask)
    regions = regionprops(labeled_mask)
    if not regions:
        print("No regions found in the binary mask.")
        return torch.from_numpy(image).float().unsqueeze(0)

    # Identify the largest connected region
    largest_region = max(regions, key=lambda r: r.area)

    # Create a mask with only the largest region filled
    diaphragm_mask = np.zeros_like(binary_mask, dtype=np.uint8)
    diaphragm_mask[labeled_mask == largest_region.label] = 255

    # Step 5: Fill any holes in the diaphragm region
    diaphragm_mask = cv2.morphologyEx(
        diaphragm_mask, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8)
    )

    # Step 6: Apply morphological closing to smooth mask (remove small holes)
    kernel = np.ones(closing_kernel_size, np.uint8)
    diaphragm_mask = cv2.morphologyEx(diaphragm_mask, cv2.MORPH_CLOSE, kernel)

    # Step 7: Bitwise operation to remove diaphragm from the source image
    result_image = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(diaphragm_mask))

    equalized_pil_image = Image.fromarray(result_image)

    return equalized_pil_image

In [3]:
preprocess_types = {
    "baseline": [resize_image],
    "histogram_equalization": [resize_image, histogram_equalization],
    "gaussian_blur": [resize_image, histogram_equalization, gaussian_blur],
    "bilateral_filer": [resize_image, histogram_equalization, bilateral_filter],
    "adaptive_masking": [resize_image, adaptive_masking],
    "adaptive_masking_equalized": [
        resize_image,
        adaptive_masking,
        histogram_equalization,
    ],
    "adaptive_masking_gaussian": [
        resize_image,
        adaptive_masking,
        histogram_equalization,
        gaussian_blur,
    ],
    "adaptive_masking_bilateral": [
        resize_image,
        adaptive_masking,
        histogram_equalization,
        bilateral_filter,
    ],
}

In [4]:
def train_model(
    model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10
):
    """Trains and validates a model for a specified number of epochs.

    Parameters
    ----------
        model: PyTorch model
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data
        criterion: Loss function
        optimizer: Optimizer
        device: Device to train on ('cuda' or 'cpu')
        num_epochs: Number of epochs

    Returns
    -------
        history: Dictionary containing training and validation loss and accuracy

    """
    model.to(device)
    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": [],
    }

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predictions = torch.max(outputs, 1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        # Validation phase
        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validation"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_loss += loss.item()
                _, predictions = torch.max(outputs, 1)
                correct += (predictions == labels).sum().item()
                total += labels.size(0)

        val_loss = running_loss / len(val_loader)
        val_acc = correct / total

        # Logging
        print(f"Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.4f}")
        print(f"Validation Loss: {val_loss:.4f} - Validation Acc: {val_acc:.4f}")
        print()

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

    return history


def test_model(model, test_loader, device):
    """Tests a model on a test set.

    Parameters
    ----------
        model: PyTorch model
        test_loader: DataLoader for test data
        device: Device to test on ('cuda' or 'cpu')

    Returns
    -------
        y_true: True labels
        y_pred: Predicted labels

    """
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            y_true += labels.tolist()
            y_pred += predictions.tolist()

        # compute accuracy
        accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [5]:
## Model pipeline
def model_pipelines(
    model,
    model_name,
    preprocess=None,
    root="dataset/new_data",
    save_path="models_pretrained",
):
    # Result table
    results = np.array([["Preprocess", "Test Accuracy"]])
    os.makedirs(save_path, exist_ok=True)

    # Loop through the preprocess_types
    for key, value in preprocess_types.items():
        functions = preprocess_types[key]
        if preprocess is not None and key not in preprocess:
            continue
        print(f"\n===== {key} =====")
        transform = transforms.Compose(
            functions
            + [
                transforms.Lambda(
                    lambda x: x.convert("L") if isinstance(x, Image.Image) else x
                ),  # convert to grayscale
                transforms.Lambda(
                    lambda x: x
                    if isinstance(x, torch.Tensor)
                    else transforms.ToTensor()(x)
                ),  # convert to tensor (To ensure torch.Size([1, 224, 224]))
                transforms.Lambda(
                    lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x
                ),  # Convert single channel to RGB (3 channels)
            ]
        )
        normal_transform = transforms.Compose(
            [resize_image]
            + [
                transforms.Lambda(
                    lambda x: x.convert("L") if isinstance(x, Image.Image) else x
                ),  # convert to grayscale
                transforms.Lambda(
                    lambda x: x
                    if isinstance(x, torch.Tensor)
                    else transforms.ToTensor()(x)
                ),  # convert to tensor (To ensure torch.Size([1, 224, 224]))
                transforms.Lambda(
                    lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x
                ),  # Convert single channel to RGB (3 channels)
            ]
        )

        augmented_train_data = datasets.ImageFolder(root=f"{root}/train", transform=transform)
        normal_train_data = datasets.ImageFolder(root=f"{root}/train", transform=normal_transform)
        test_data = datasets.ImageFolder(root=f"{root}/test", transform=normal_transform)
        val_data = datasets.ImageFolder(root=f"{root}/val", transform=normal_transform)
        train_data = ConcatDataset([normal_train_data, augmented_train_data])

        # Extract labels from the normal and augmented datasets
        normal_targets = [sample[1] for sample in normal_train_data.imgs]
        augmented_targets = [sample[1] for sample in augmented_train_data.imgs]

        # Combine the labels
        targets = normal_targets + augmented_targets

        # Apply oversampling using imblearn
        sampler = RandomOverSampler(random_state=42)
        indices = list(range(len(targets)))
        resampled_indices, _ = sampler.fit_resample(np.array(indices).reshape(-1, 1), targets)
        resampled_indices = resampled_indices.flatten()

        # Create resampled dataset
        resampled_dataset = Subset(train_data, resampled_indices)
        # Print original and resampled class distributions
        print(f"Original class distribution: {Counter(targets)}")
        resampled_labels = [train_data[idx][1] for idx in resampled_indices]
        print(f"Resampled class distribution: {Counter(resampled_labels)}")

        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

        print(len(train_loader))
        # raise Warning("Stop")

        # Initialize the model
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # Train the model
        history = train_model(
            model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10
        )

        # Evaluate the model
        accuracy = test_model(model, test_loader, device)
        torch.save(model, f"{save_path}/{model_name}_{key}.pth")
        results = np.append(results, [[key, accuracy]], axis=0)
        print(f"Test Accuracy: {accuracy}")
        print("\n")

    return results

In [6]:
## Define the CNN model
class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.dropout2 = nn.Dropout(0.25)

        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.dropout3 = nn.Dropout(0.4)

        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 3)  # Three classes: NORMAL, BACTERIA, VIRUS

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = self.dropout2(x)

        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool(x)
        x = self.dropout3(x)

        x = x.view(-1, 128 * 28 * 28)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.dropout4(x)
        x = self.fc2(x)
        return x


In [7]:
# Use the oversampled dataset
root="dataset/new_data"
# DenseNet161 with adaptive_masking_bilateral
print("\n=========================\nDenseNet161 with adaptive_masking_bilateral\n=========================")
model = models.densenet161(pretrained=True)
results = model_pipelines(model, "DenseNet161", root=root, preprocess=["adaptive_masking_bilateral"])
print(results)

# EfficientNetB1 with adaptive_masking_gaussian
print("\n=========================\nEfficientNetB1 with adaptive_masking_gaussian\n=========================")
model = models.efficientnet_b1(pretrained=True)
results = model_pipelines(model, "EfficientNetB1", root=root, preprocess=["adaptive_masking_gaussian"])
print(results)

# ResNet50 with adaptive_masking_equalized
print("\n=========================\nResNet50 with adaptive_masking_equalized\n=========================")
model = models.resnet50(pretrained=True)
results = model_pipelines(model, "ResNet50", root=root, preprocess=["adaptive_masking_equalized"])
print(results)

# VGG16 with adaptive_masking
print("\n=========================\nVGG16 with gaussian_blur\n=========================")
model = models.vgg16(pretrained=True)
results = model_pipelines(model, "VGG16", root=root, preprocess=["gaussian_blur"])
print(results)


DenseNet161 with adaptive_masking_bilateral


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:00<00:00, 209MB/s]  



===== adaptive_masking_bilateral =====
Original class distribution: Counter({0: 4676, 1: 2298, 2: 2290})
Resampled class distribution: Counter({0: 4676, 1: 4676, 2: 4676})
290
Epoch 1/10


Validation: 100%|██████████| 19/19 [00:13<00:00,  1.46it/s]


Train Loss: 0.9669 - Train Acc: 0.5928
Validation Loss: 1.1932 - Validation Acc: 0.3333

Epoch 2/10


...

Train Loss: 1.0402 - Train Acc: 0.5047
Validation Loss: 1.1720 - Validation Acc: 0.3333

Epoch 9/10


Validation: 100%|██████████| 19/19 [00:11<00:00,  1.64it/s]


Train Loss: 1.0407 - Train Acc: 0.5047
Validation Loss: 1.2012 - Validation Acc: 0.3333

Epoch 10/10


Validation: 100%|██████████| 19/19 [00:11<00:00,  1.64it/s]


Train Loss: 1.0425 - Train Acc: 0.5047
Validation Loss: 1.1360 - Validation Acc: 0.3333

Test Accuracy: 0.38782051282051283


[['Preprocess' 'Test Accuracy']
 ['gaussian_blur' '0.38782051282051283']]


In [ ]:
# Data for the comparison table of the four models with their preprocess techniques and test accuracies
model_comparison_data = {
    "Model & Preprocess": [
        "DenseNet161 with adaptive_masking_bilateral",
        "EfficientNetB1 with adaptive_masking_gaussian",
        "ResNet50 with adaptive_masking_equalized",
        "VGG16 with gaussian_blur"
    ],
    "Test Accuracy": [
        "0.38782051282051283",
        "0.6105769230769231",
        "0.5753205128205128",
        "0.38782051282051283"
    ]
}

# Creating the comparison DataFrame
model_comparison_df = pd.DataFrame(model_comparison_data)
model_comparison_df["Test Accuracy"] = model_comparison_df["Test Accuracy"].astype(float) * 100
model_comparison_df.to_csv("results/3_oversampling_while_preserve_ori_data.csv", index=False)

model_comparison_df = model_comparison_df.round(2)
model_comparison_df

,Model & Preprocess,Test Accuracy
0,DenseNet161 with adaptive_masking_bilateral,38.78
1,EfficientNetB1 with adaptive_masking_gaussian,61.06
2,ResNet50 with adaptive_masking_equalized,57.53
3,VGG16 with gaussian_blur,38.78
